In [4]:
import json #will be needed for saving preprocessing details
import numpy as np # for data manipulation
import pandas as pd # for data manipulation
from sklearn.model_selection import train_test_split # will be used for data split
from sklearn.preprocessing import LabelEncoder # for preprocessing
from sklearn.ensemble import RandomForestClassifier # for training the algorithm
from sklearn.ensemble import ExtraTreesClassifier # for training the algorithm
import joblib # for saving algorithm and preprocessing objects
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import learning_curve

In [6]:
data= pd.read_csv(r'C:\Users\Michael Owen\OneDrive\Desktop\Datasets\5percent_MSSQL.csv')
#Load dataset
data.head(4)

c:\users\michael owen\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (86) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,Unnamed: 0.1,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
0,568269,42641,172.16.0.5-192.168.50.4-55989-50529-17,172.16.0.5,55989,192.168.50.4,50529,17,2018-11-03 10:34:48.345746,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,MSSQL
1,3914458,89976,172.16.0.5-192.168.50.4-61850-36734-17,172.16.0.5,61850,192.168.50.4,36734,17,2018-11-03 10:40:17.959592,50,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,MSSQL
2,2789854,85749,172.16.0.5-192.168.50.4-39757-45349-17,172.16.0.5,39757,192.168.50.4,45349,17,2018-11-03 10:37:47.706365,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,MSSQL
3,2834358,86263,172.16.0.5-192.168.50.4-11300-20633-17,172.16.0.5,11300,192.168.50.4,20633,17,2018-11-03 10:37:51.304213,49,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,MSSQL


In [7]:
data.shape

(288789, 89)

In [8]:
original_features=list(data.columns)
len(original_features)

89

In [9]:
features_missing_values=list(data.columns[data.isna().any()])
len(features_missing_values)

1

In [10]:
categorical_features=list(data.select_dtypes(include=['object']).columns)
categorical_features

['Flow ID',
 'Source IP',
 'Destination IP',
 'Timestamp',
 'SimillarHTTP',
 'Label']

In [11]:
numerical=list(set(original_features)-set(categorical_features))
numerical

['Flow IAT Min',
 'Bwd Avg Bulk Rate',
 'Fwd Packet Length Std',
 'Fwd Packet Length Min',
 'Fwd PSH Flags',
 'Fwd Avg Packets/Bulk',
 'Bwd Packet Length Min',
 'Bwd Packet Length Max',
 'CWE Flag Count',
 'Average Packet Size',
 'Protocol',
 'Flow IAT Std',
 'Packet Length Variance',
 'Idle Min',
 'Fwd URG Flags',
 'Bwd Packets/s',
 'Max Packet Length',
 'min_seg_size_forward',
 'Bwd Packet Length Mean',
 'Bwd IAT Total',
 'Flow IAT Mean',
 'Idle Max',
 'Total Backward Packets',
 'ACK Flag Count',
 'Fwd IAT Min',
 'Packet Length Mean',
 'Active Max',
 'Fwd Packet Length Max',
 'Fwd Header Length',
 'Bwd IAT Min',
 'Total Fwd Packets',
 'Packet Length Std',
 'PSH Flag Count',
 'URG Flag Count',
 'Unnamed: 0',
 'Fwd Packets/s',
 'Subflow Fwd Bytes',
 'Fwd Header Length.1',
 'FIN Flag Count',
 'Idle Std',
 'Total Length of Bwd Packets',
 'Init_Win_bytes_backward',
 'Flow Bytes/s',
 'Active Mean',
 'Inbound',
 'ECE Flag Count',
 'Active Min',
 'Fwd Packet Length Mean',
 'RST Flag Count',


In [12]:
encoders = {}
for column in ['Label']:
    categorical_convert = LabelEncoder()

In [13]:
nominal=['Label']
ordinal=list(set(categorical_features)-set(nominal))

In [14]:
df_nominal=pd.get_dummies(data[nominal])

In [15]:
target=['Label']

In [16]:
for feature in ordinal:
  data[feature]=data[feature].astype('category').cat.codes

df_ordinal=data[ordinal]

In [17]:
data[numerical]

,Flow IAT Min,Bwd Avg Bulk Rate,Fwd Packet Length Std,Fwd Packet Length Min,Fwd PSH Flags,Fwd Avg Packets/Bulk,Bwd Packet Length Min,Bwd Packet Length Max,CWE Flag Count,Average Packet Size,...,Unnamed: 0.1,Source Port,Bwd URG Flags,Destination Port,Flow Duration,act_data_pkt_fwd,Init_Win_bytes_forward,Fwd Avg Bulk Rate,Idle Mean,Flow IAT Max
0,1.0,0,0.0,444.0,0,0,0.0,0.0,0,666.0,...,42641,55989,0,50529,1,1,-1,0,0.0,1.0
1,50.0,0,0.0,431.0,0,0,0.0,0.0,0,646.5,...,89976,61850,0,36734,50,1,-1,0,0.0,50.0
2,2.0,0,0.0,554.0,0,0,0.0,0.0,0,831.0,...,85749,39757,0,45349,2,1,-1,0,0.0,2.0
3,49.0,0,0.0,476.0,0,0,0.0,0.0,0,714.0,...,86263,11300,0,20633,49,1,-1,0,0.0,49.0
4,1.0,0,0.0,1100.0,0,0,0.0,0.0,0,1650.0,...,25038,61850,0,3134,1,1,-1,0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288784,1.0,0,0.0,490.0,0,0,0.0,0.0,0,735.0,...,7029,17471,0,51197,1,1,-1,0,0.0,1.0
288785,1.0,0,0.0,444.0,0,0,0.0,0.0,0,666.0,...,121994,16055,0,33609,1,1,-1,0,0.0,1.0
288786,1.0,0,0.0,494.0,0,0,0.0,0.0,0,741.0,...,9448,32855,0,6197,1,1,-1,0,0.0,1.0
288787,1.0,0,0.0,1472.0,0,0,0.0,0.0,0,2208.0,...,24043,15143,0,14340,1,1,-1,0,0.0,1.0


In [18]:
new_data=pd.concat([df_nominal,df_ordinal,data[numerical]],axis=1)
new_data.shape

(288789, 91)